In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
df = pd.read_csv('UpdatedResumeDataSet.csv')
df.head()

In [ ]:
df.shape

# Exploring Categories

In [ ]:
df['Category'].value_counts()

In [ ]:
print(df['Category'].dtype)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15, 5))

# Visualizing counts of categories
sns.countplot(data=df, x='Category')  # Assuming 'Category' is the column name
plt.xticks(rotation=45)  # Rotating x-axis labels for better readability
plt.show()

In [ ]:
# making pie plot .
plt.figure(figsize=(15, 7))
counts = df['Category'].value_counts()
labels = df['Category'].unique()
plt.pie(counts,labels=labels,autopct='%1.1f%%',shadow=True,
        colors=plt.cm.coolwarm(np.linspace(0,1,3)))
plt.show()

# Exploring Resume

In [ ]:
df['Category'][0]

In [ ]:
df['Resume'][0]

# Cleaning Data
1 : URLs,

2 : Hashtags,

3 : mentions,

4 : special letters,

5 : punctuatuions

In [ ]:
import re 
def cleanResume(txt):
#     .)The blw remove the http and |s+ means all char combine with it.
    cleanTxt = re.sub('http\S+',' ',txt)
    cleanTxt = re.sub('@\S+',' ',cleanTxt)
    cleanTxt = re.sub('#\S+',' ',cleanTxt)
    
#     .)The blw is trick of removing all special symbols.
    cleanTxt = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_{|}~  """),' ',cleanTxt)
    
#     .)The blw is speical text fing in it which is observed by checking the ext.
    cleanTxt = re.sub(r'[^\x00-\x7f]',' ',cleanTxt)
# The line re.sub('\s+', ' ', cleanTxt) essentially replaces sequences of 
# one or more whitespace characters in the string variable cleanTxt with
# a single whitespace.
    cleanTxt = re.sub('\s+',' ',cleanTxt)
    return cleanTxt

In [ ]:
cleanResume('My website subhan@gmail #name is ###  http://subhan.com and its name')

In [ ]:
df['Resume'] = df['Resume'].apply(lambda x :cleanResume(x) )

In [ ]:
df['Resume'][0]

# categorical values into numerical form.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
le.fit(df['Category'])
df['Category'] = le.transform(df['Category'])

In [ ]:
df.head()

# TFIDF vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(df['Resume'])
requiredTaxt = tfidf.transform(df['Resume'])

# splitting

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(requiredTaxt,df['Category'],test_size=0.2,
                                                 random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

# Applying K nearest alogorith for classification

In [ ]:
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.multiclass import OneVsRestClassifier 
from sklearn.metrics import accuracy_score
# .)The blw OneVsRestClassifier is used it is used when ther eare multiclass labels in 
# y so to improve accuracy we use OneVsRestClassifier it.
clf = OneVsRestClassifier(KNeighborsClassifier())
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test,y_pred))

# predictive system

In [ ]:
myresume= """
John Doe
johndoe@email.com
123-456-7890
123 Main Street, City, Country

Summary:
A highly motivated individual with experience in software development and data analysis. Strong problem-solving skills and ability to work in a team environment.

Education:
- Bachelor of Science in Computer Science
  University of Example, 2022

- High School Diploma
  Example High School, 2018

Experience:
- Software Engineer Intern
  Tech Company X, 2021 - 2022
  - Developed and maintained software applications.
  - Collaborated with the team to improve project efficiency.

- Customer Service Representative
  Service Provider Inc., 2019 - 2021
  - Assisted customers with inquiries and complaints.
  - Handled customer service issues effectively.

Skills:
- Python
- Java
- Data Analysis
- Communication"""

In [ ]:
# using pickle to save model and vectorizer.
import pickle 
pickle.dump(tfidf,open('tfidf.pkl','wb'))
pickle.dump(clf,open('clf.pkl','wb'))

In [ ]:
# .)Now using the save model and vectorizer by loading it .
import pickle

#loading the trained model classifier
clf = pickle.load(open('clf.pkl','rb'))

#clean the input resume.
cleaned_resume = cleanResume(myresume)

# Transform the cleaned resume using the tfidf vector.
input_features = tfidf.transform([cleaned_resume])

# making the prediction using the loaded model.
predictions_id = clf.predict(input_features)[0]
# predictions_id
# map category id to category name.
p = le.inverse_transform([predictions_id])[0]
print("Your resume is showing that your field is :",p)